In [ ]:
from hdx.utilities.easy_logging import setup_logging
from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import re
import itertools 
import pickle
from sklearn.neural_network import MLPClassifier
from fastText import load_model
from sklearn.model_selection import train_test_split
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
df = pd.read_csv("try.csv")
df.head()

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopWords = set(stopwords.words('english'))

In [ ]:
fasttext_model = 'wiki.en.bin'
fmodel = load_model(fasttext_model)

In [ ]:
def lower_cols(lst):
    return [word.lower() for word in lst]


def remove_chars(lst):

    cleaned = [re.sub('\s+', ' ', mystring).strip() for mystring in lst]
    cleaned = [re.sub(r'[[^A-Za-z0-9\s]+]', ' ', mystr) for mystr in cleaned]
    cleaned = [mystr.replace('_', ' ') for mystr in cleaned]
    return cleaned

def clean_cols(data):
    data = lower_cols(data)
    data = remove_chars(data)
    return data

In [ ]:
def preprocess(pandas_dataset, df_target):
    if (not pandas_dataset.empty):
    	organization = 'HDX'   #Replace if datasets contains organization
    	headers = list(pandas_dataset.columns.values)
    	headers = clean_cols(headers)
    for i in range(len(headers)):
        try:
            dic = {'Header': headers[i], 
                   'Data': list(pandas_dataset.iloc[1:, i]), 
                   'Relative Column Position': (i+1) / len(pandas_dataset.columns), 
                   'Organization': organization,
                   'Index': i}
            df_target.loc[len(df_target)] = dic
        except:
            raise Exception("Error: arguments not matched")

    df_result = transform_vectorizers(df_target)
    return df_result

def transform_vectorizers(df_target):
    cols = ['Header_embedding', 'Organization_embedded', 'features_combined']
    df = pd.DataFrame(columns = cols)
    df = df.dropna()
    df_target = df_target.dropna()
    df_target = embedded_datapoints(df_target)
    df['data_combined'] = df_target.loc[:, 'embedded_datapoint0': 'embedded_datapoint' 
                                                           + str(number_of_data_point_to_vectorize-1)].values.tolist()
    df['data_combined'] = df['data_combined'].apply(lambda x: [val for item in x for val in item])
    df['Header_embedding'] = df_target['Header'].astype(str).apply(fmodel.get_sentence_vector)
    df['Organization_embedded'] = df_target['Organization'].astype(str).apply(fmodel.get_sentence_vector)
    cols = ['Header_embedding', 'Organization_embedded', 'data_combined']
    df['features_combined'] = df[cols].values.tolist()
    df['features_combined'] = df['features_combined'].apply(lambda x: [val for item in x for val in item])
    diff = 2934 - len(df['features_combined'][0])
    for i in range(len(df)):
        for j in range(diff):
            df['features_combined'][i].append(0)
    df = df.dropna()
    return df

In [ ]:
def remove_stop_words(data_lst):
    #remove stopwords from the data including 'the', 'and' etc.
    wordsFiltered = []
    for w in data_lst:
        if w not in stopWords:
            wordsFiltered.append(w)
    return wordsFiltered

def word_extract(row):
    ignore = ['nan']
    no_white = [i.lstrip() for i in row if i not in ignore and not isinstance(i, float)]
    cleaned_text = [w.lower() for w in no_white if w not in ignore]
    return cleaned_text

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS 

from nltk import ngrams
def generate_n_grams(data_lst, n):
    # cleaned = remove_chars(list(data_lst))
    # cleaned = clean_cols(cleaned)
    cleaned = remove_stop_words(data_lst)
    #make sure that n_grams 'refresh' when a new dataset is encountered!!!!   
    return list(ngrams(cleaned, n))

In [ ]:
def separate_words(series): 
    #each series is a long string that contains all the data
    lst = []
    for i in series:
        print(i)
        lst = re.split(r"\W+", i)
       
        lst.extend(list(filter(None, lst)))
    return lst
    
def vectorize_n_datapoints(df, number_of_datapoints_to_vectorize = 7):
    print(df['Data'].head())
    df['Data_separated'] = df['Data'].apply(separate_words)
    if (number_of_datapoints_to_vectorize > len(df['Data_separated'][0])):
        number_of_datapoints_to_vectorize = len(df['Data_separated'][0])
    for i in range(number_of_datapoints_to_vectorize):
        df['datapoint' + str(i)] = df['Data_separated'].str[i]
    return df  
def embedded_datapoints(df, number_of_data_point_to_vectorize=7):
    df = vectorize_n_datapoints(df)
    for i in range(number_of_data_point_to_vectorize):
        df['embedded_datapoint' + str(i)] = df['datapoint' + str(i)].map(lambda x: fmodel.get_sentence_vector(str(x)))
    return df

In [ ]:
processed_dataset = preprocess(df, 
                pd.DataFrame(columns=['Header','Data','Relative Column Position','Organization','Index']))

In [ ]:
len(processed_dataset)

In [ ]:
processed_dataset['features_combined'][1][626]

In [ ]:
# diff = 2934 - len(processed_dataset['features_combined'][0])
# for i in range(len(processed_dataset)):
#     for j in range(diff):
#         processed_dataset['features_combined'][i].append(0)
        
# len(processed_dataset['features_combined'][6])

In [ ]:
model = pickle.load(open("model.pkl", "rb"))

In [ ]:
model.predict(list(processed_dataset['features_combined']))